# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 1:44PM,261231,19,ADV80015944,"AdvaGen Pharma, Ltd",Released
1,Apr 17 2023 1:44PM,261231,19,ADV80015945,"AdvaGen Pharma, Ltd",Released
2,Apr 17 2023 1:44PM,261231,19,ADV80015946,"AdvaGen Pharma, Ltd",Released
3,Apr 17 2023 1:44PM,261231,19,ADV80015947,"AdvaGen Pharma, Ltd",Released
4,Apr 17 2023 1:44PM,261231,19,ADV80015948,"AdvaGen Pharma, Ltd",Released
5,Apr 17 2023 1:44PM,261231,19,ADV80015949,"AdvaGen Pharma, Ltd",Released
6,Apr 17 2023 1:44PM,261231,19,ADV80015950,"AdvaGen Pharma, Ltd",Released
7,Apr 17 2023 1:44PM,261231,19,ADV80015951,"AdvaGen Pharma, Ltd",Released
8,Apr 17 2023 1:44PM,261231,19,ADV80015952,"AdvaGen Pharma, Ltd",Released
9,Apr 17 2023 1:44PM,261231,19,ADV80015953,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,261223,Released,7
22,261224,Released,3
23,261226,Released,21
24,261230,Released,1
25,261231,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261223,NaN,7.0
261224,NaN,3.0
261226,NaN,21.0
261230,NaN,1.0
261231,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261146,0.0,1.0
261147,1.0,0.0
261161,0.0,1.0
261172,1.0,0.0
261179,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261146,0,1
261147,1,0
261161,0,1
261172,1,0
261179,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261146,0,1
1,261147,1,0
2,261161,0,1
3,261172,1,0
4,261179,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261146,,1
1,261147,1,
2,261161,,1
3,261172,1,
4,261179,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd"
13,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc"
14,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc."
35,Apr 17 2023 1:15PM,261223,10,Hush Hush
42,Apr 17 2023 1:12PM,261224,10,"Snap Medical Industries, LLC"
45,Apr 17 2023 1:12PM,261222,10,Emerginnova
46,Apr 17 2023 1:06PM,261220,10,"Methapharm, Inc."
51,Apr 17 2023 12:48PM,261218,10,"NBTY Global, Inc."
52,Apr 17 2023 12:46PM,261217,10,Eywa Pharma Inc.
53,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",,13
1,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",,1
2,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",,21
3,Apr 17 2023 1:15PM,261223,10,Hush Hush,,7
4,Apr 17 2023 1:12PM,261224,10,"Snap Medical Industries, LLC",,3
5,Apr 17 2023 1:12PM,261222,10,Emerginnova,,1
6,Apr 17 2023 1:06PM,261220,10,"Methapharm, Inc.",,5
7,Apr 17 2023 12:48PM,261218,10,"NBTY Global, Inc.",,1
8,Apr 17 2023 12:46PM,261217,10,Eywa Pharma Inc.,,1
9,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",13,
1,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",1,
2,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",21,
3,Apr 17 2023 1:15PM,261223,10,Hush Hush,7,
4,Apr 17 2023 1:12PM,261224,10,"Snap Medical Industries, LLC",3,
5,Apr 17 2023 1:12PM,261222,10,Emerginnova,1,
6,Apr 17 2023 1:06PM,261220,10,"Methapharm, Inc.",5,
7,Apr 17 2023 12:48PM,261218,10,"NBTY Global, Inc.",1,
8,Apr 17 2023 12:46PM,261217,10,Eywa Pharma Inc.,1,
9,Apr 17 2023 12:11PM,261209,12,"SD Head USA, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",13,
1,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",1,
2,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",21,
3,Apr 17 2023 1:15PM,261223,10,Hush Hush,7,
4,Apr 17 2023 1:12PM,261224,10,"Snap Medical Industries, LLC",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",13.0,NaN
1,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",1.0,NaN
2,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",21.0,NaN
3,Apr 17 2023 1:15PM,261223,10,Hush Hush,7.0,NaN
4,Apr 17 2023 1:12PM,261224,10,"Snap Medical Industries, LLC",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 17 2023 1:44PM,261231,19,"AdvaGen Pharma, Ltd",13.0,0.0
1,Apr 17 2023 1:36PM,261230,10,"Uniderm USA, Inc",1.0,0.0
2,Apr 17 2023 1:19PM,261226,20,"Exact-Rx, Inc.",21.0,0.0
3,Apr 17 2023 1:15PM,261223,10,Hush Hush,7.0,0.0
4,Apr 17 2023 1:12PM,261224,10,"Snap Medical Industries, LLC",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3395730,85.0,13.0
12,261209,1.0,0.0
16,261194,1.0,0.0
18,261161,1.0,0.0
19,1828275,17.0,2.0
20,261226,21.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3395730,85.0,13.0
1,12,261209,1.0,0.0
2,16,261194,1.0,0.0
3,18,261161,1.0,0.0
4,19,1828275,17.0,2.0
5,20,261226,21.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,85.0,13.0
1,12,1.0,0.0
2,16,1.0,0.0
3,18,1.0,0.0
4,19,17.0,2.0
5,20,21.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,85.0
1,12,Released,1.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Executing,13.0,0.0,0.0,0.0,2.0,0.0
Released,85.0,1.0,1.0,1.0,17.0,21.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Executing,13.0,0.0,0.0,0.0,2.0,0.0
1,Released,85.0,1.0,1.0,1.0,17.0,21.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Executing,13.0,0.0,0.0,0.0,2.0,0.0
1,Released,85.0,1.0,1.0,1.0,17.0,21.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()